# Тестирование модулей обработки писем

Проверка всех функций реализованных модулей на 8 письмах:
- 2 фишинговых из датасета
- 2 легитимных из датасета  
- 2 тестовых легитимных
- 2 тестовых фишинговых


In [ ]:
import sys
from pathlib import Path
import pandas as pd

BASE_DIR = Path('../').resolve()
sys.path.insert(0, str(BASE_DIR))

# Настройка путей
DATA_RAW = BASE_DIR / 'data' / 'raw'
DATA_PROCESSED = BASE_DIR / 'data' / 'processed'
TEST_EMAILS = BASE_DIR / 'test_emails'
TI_DB = BASE_DIR / 'data' / 'threat_intelligence' / 'ti_test_real_feeds.db'

from src.email_parser import load_eml_file, parse_email
from src.translation import Translator
from src.feature_extractor import FeatureExtractor
from src.header_analyzer import analyze_headers
from src.url_domain_analyzer import analyze_urls_and_domains
from src.threat_intelligence import ThreatIntelligence
from src.rules_engine import evaluate_all_rules


In [11]:
# Загрузка датасета для получения писем 
df = pd.read_csv(DATA_PROCESSED / 'email_dataset.csv')

# Выбираем письма для тестирования
test_emails = {
    'dataset_phishing_1': df.iloc[0]['email_content'],
    'dataset_phishing_2': df.iloc[80]['email_content'],
    'dataset_legit_1': df.iloc[-2]['email_content'],
    'dataset_legit_2': df.iloc[-1]['email_content'],
    'test_legit_1': load_eml_file(TEST_EMAILS / 'legitimate_01_sberbank_ru.eml'),
    'test_legit_2': load_eml_file(TEST_EMAILS / 'legitimate_02_mvideo_ru.eml'),
    'test_phish_1': load_eml_file(TEST_EMAILS / 'suspicious_01_phishing.eml'),
    'test_phish_2': load_eml_file(TEST_EMAILS / 'suspicious_02_phishing.eml')
}


## 1. Тестирование email_parser


In [ ]:
# Полный вывод всех данных, извлеченных email_parser для каждого тестового письма
import json
from pprint import pprint

print("=" * 80)
print("Проверка модуля email_parser")
print("=" * 80)

for email_name, email_content in test_emails.items():
    print(f"\n{'='*80}")
    print(f"ПИСЬМО: {email_name}")
    print(f"{'='*80}\n")
    
    try:
        parsed = parse_email(email_content)
        
        # Выводим все данные
        print("ЗАГОЛОВКИ:")
        print(f"  From: {parsed.get('from', 'N/A')}")
        print(f"  To: {parsed.get('to', 'N/A')}")
        print(f"  Subject: {parsed.get('subject', 'N/A')}")
        print(f"  Date: {parsed.get('date', 'N/A')}")
        print(f"  Message-ID: {parsed.get('message_id', 'N/A')}")
        print(f"  Reply-To: {parsed.get('reply_to', 'N/A')}")
        print(f"  Return-Path: {parsed.get('return_path', 'N/A')}")
        print(f"  References: {parsed.get('references', 'N/A')}")
        print(f"  Authentication-Results: {parsed.get('auth_results', 'N/A')}")
        
        print(f"\nRECEIVED HEADERS ({len(parsed.get('received_headers', []))}):")
        received = parsed.get('received_headers', [])
        if received:
            for i, rec in enumerate(received, 1):
                print(f"  [{i}] {rec[:200]}{'...' if len(str(rec)) > 200 else ''}")
        else:
            print("  (нет данных)")
        
        print(f"\n ТЕЛО ПИСЬМА:")
        body_plain = parsed.get('body_plain', '')
        body_html = parsed.get('body_html', '')
        print(f"  Plain text длина: {len(body_plain)} символов")
        if body_plain:
            print(f"  Plain text (первые 300 символов): {body_plain[:300]}{'...' if len(body_plain) > 300 else ''}")
        else:
            print("  Plain text: (пусто)")
        print(f"  HTML длина: {len(body_html)} символов")
        if body_html:
            print(f"  HTML (первые 300 символов): {body_html[:300]}{'...' if len(body_html) > 300 else ''}")
        else:
            print("  HTML: (пусто)")
        
        print(f"\n ВЛОЖЕНИЯ ({len(parsed.get('attachments', []))}):")
        attachments = parsed.get('attachments', [])
        if attachments:
            for i, att in enumerate(attachments, 1):
                print(f"  [{i}] {att.get('name', 'unknown')}")
                print(f"      Тип: {att.get('type', 'N/A')}")
                print(f"      Размер: {att.get('size', 0)} байт")
                print(f"      SHA-256: {att.get('sha256', 'N/A')}")
        else:
            print("  (нет вложений)")
        
        print(f"\n URL ({len(parsed.get('urls', []))}):")
        urls = parsed.get('urls', [])
        if urls:
            for i, url in enumerate(urls, 1):
                print(f"  [{i}] {url}")
        else:
            print("  (нет URL)")
        
        print(f"\n ДОМЕНЫ ({len(parsed.get('domains', []))}):")
        domains = parsed.get('domains', [])
        if domains:
            for i, domain in enumerate(domains, 1):
                print(f"  [{i}] {domain}")
        else:
            print("  (нет доменов)")
        
        print(f"\n  IP-АДРЕСА ({len(parsed.get('ips', []))}):")
        ips = parsed.get('ips', [])
        if ips:
            for i, ip in enumerate(ips, 1):
                print(f"  [{i}] {ip}")
        else:
            print("  (нет IP-адресов)")
        
        print(f"\n СТАТИСТИКА:")
        print(f"  Всего URL: {len(urls)}")
        print(f"  Всего доменов: {len(domains)}")
        print(f"  Всего IP: {len(ips)}")
        print(f"  Всего вложений: {len(attachments)}")
        print(f"  Длина plain text: {len(body_plain)}")
        print(f"  Длина HTML: {len(body_html)}")
        print(f"  Количество Received headers: {len(received)}")
        
    except Exception as e:
        print(f" ОШИБКА при обработке {email_name}: {e}")
        import traceback
        traceback.print_exc()
    
    print("\n")


Проверка модуля email_parser

ПИСЬМО: dataset_phishing_1

ЗАГОЛОВКИ:
  From: "monkey.org Account Policy Server | This Message was Sent on Behalf Of monkey.org Account Policy Admin" <contact@igse.com.mx>
  To: jose@monkey.org
  Subject: Account Policy and Important Account Notification from monkey.org 
  Date: Thu, 17 Aug 2023 04:25:45 -0700
  Message-ID: <20230817042545.B909E93B20818E00@igse.com.mx>
  Reply-To: support@igse.com.mx
  Return-Path: contact@igse.com.mx
  References: 
  Authentication-Results: imf09.b.hostedemail.com;	dkim=pass header.d=igse.com.mx header.s=default header.b=XKpA0wcR;	spf=pass (imf09.b.hostedemail.com: domain of contact@igse.com.mx designates 185.161.210.150 as permitted sender) smtp.mailfrom=contact@igse.com.mx;	dmarc=pass (policy=reject) header.from=igse.com.mx

RECEIVED HEADERS (1):
  [1] from dns0.igse.com.mx (dns0.igse.com.mx [185.161.210.150])	by imf09.b.hostedemail.com (Postfix) with ESMTP id D908F180008	for <jose@monkey.org>; Thu, 17 Aug 2023 11:41:2

## 2. Тестирование translation


In [ ]:
print("=" * 80)
print("Проверка модуля translation")
print("=" * 80)

translator = Translator()

for name, email_content in test_emails.items():
    try:
        parsed = parse_email(email_content)
        
        # Подготовка текста из parsed email (subject + body, очищенный от HTML)
        prepared_text = FeatureExtractor.prepare_text_from_parsed_email(parsed)
        
        # Определение языка исходного текста
        source_lang = translator.detect_language(prepared_text if prepared_text else parsed.get('subject', ''))
        
        # Перевод текста на английский (если требуется)
        translated_text = translator.translate_text(prepared_text)
        
        # Определение языка переведенного текста
        translated_lang = translator.detect_language(translated_text if translated_text else prepared_text)
        
        print(f"{name}: source_lang={source_lang}, translated_lang={translated_lang}, "
              f"prepared_len={len(prepared_text)}, translated_len={len(translated_text)}")
    except Exception as e:
        print(f"{name}: ОШИБКА - {e}")
        import traceback
        traceback.print_exc()


Проверка модуля translation
dataset_phishing_1: source_lang=en, translated_lang=en, prepared_len=468, translated_len=468
dataset_phishing_2: source_lang=en, translated_lang=en, prepared_len=48, translated_len=48
dataset_legit_1: source_lang=en, translated_lang=en, prepared_len=1079, translated_len=1079
dataset_legit_2: source_lang=en, translated_lang=en, prepared_len=3520, translated_len=3520
test_legit_1: source_lang=ru, translated_lang=en, prepared_len=1022, translated_len=1025
test_legit_2: source_lang=ru, translated_lang=en, prepared_len=941, translated_len=984
test_phish_1: source_lang=ru, translated_lang=en, prepared_len=1145, translated_len=1063
test_phish_2: source_lang=ru, translated_lang=en, prepared_len=978, translated_len=1030


## 3. Тестирование header_analyzer


In [14]:
for name, email_content in test_emails.items():
    parsed = parse_email(email_content)
    headers = {
        'from': parsed['from'],
        'to': parsed['to'],
        'subject': parsed['subject'],
        'auth_results': parsed['auth_results'],
        'reply_to': parsed['reply_to'],
        'return_path': parsed['return_path'],
        'received_headers': parsed['received_headers'],
        'references': parsed['references']
    }
    header_analysis = analyze_headers(headers)
    print(f"{name}: spf={header_analysis['spf_result']}, dkim={header_analysis['dkim_result']}, from_domain={header_analysis['from_domain']}")


dataset_phishing_1: spf=pass, dkim=pass, from_domain=igse.com.mx
dataset_phishing_2: spf=none, dkim=none, from_domain=us.af.mil
dataset_legit_1: spf=none, dkim=none, from_domain=bradfordcompany.com
dataset_legit_2: spf=none, dkim=none, from_domain=motleyfool.com
test_legit_1: spf=pass, dkim=pass, from_domain=sberbank.ru
test_legit_2: spf=pass, dkim=pass, from_domain=mvideo.ru
test_phish_1: spf=fail, dkim=fail, from_domain=sberbank.ru
test_phish_2: spf=softfail, dkim=fail, from_domain=technoservice.ru


## 4. Тестирование url_domain_analyzer


In [19]:
for name, email_content in test_emails.items():
    parsed = parse_email(email_content)
    url_analysis = analyze_urls_and_domains(parsed)
    print(f"{name}: {url_analysis}")

print("\n" + "=" * 80)
print("Тестирование на искусственных доменах")
print("=" * 80)

from src.url_domain_analyzer import (
    is_long_domain, is_suspicious_tld, 
    detect_url_shorteners, detect_ip_in_url,
    has_shortened_url, has_ip_based_url
)

# Тестовые домены и URL
test_domains = {
    'normal': 'example.com',
    'long': '1234567890verylongdomainname123456789.com',
    'suspicious_xin': 'test.xin',
    'suspicious_win': 'example.win',
    'suspicious_top': 'domain.top',
    'normal_ru': 'sberbank.ru'
}

test_urls = {
    'shortener_bitly': 'https://bit.ly/abc123',
    'shortener_tinyurl': 'https://tinyurl.com/test',
    'ip_public': 'http://185.161.210.150/login',
    'ip_private': 'http://192.168.1.1/page',
    'normal': 'https://example.com/page'
}

print("\nПроверка доменов:")
for name, domain in test_domains.items():
    long_check = is_long_domain(domain)
    suspicious_check = is_suspicious_tld(domain)
    print(f"  {name} ({domain}): подозрительная длина={long_check}, подозрительный TLD={suspicious_check}")

print("\nПроверка URL:")
for name, url in test_urls.items():
    shortener_found, shortener_domain, shortener_details = detect_url_shorteners(url)
    ip_found, ip_addr, ip_details = detect_ip_in_url(url)
    print(f"  {name} ({url}):")
    print(f"    URL shortener: {shortener_found} ({shortener_domain if shortener_found else 'нет'})")
    print(f"    IP в URL (внешний): {ip_found} ({ip_addr if ip_found else 'нет'})")


dataset_phishing_1: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': False, 'has_ip_in_url': False}
dataset_phishing_2: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': False, 'has_ip_in_url': False}
dataset_legit_1: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': False, 'has_ip_in_url': False}
dataset_legit_2: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': False, 'has_ip_in_url': False}
test_legit_1: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': False, 'has_ip_in_url': False}
test_legit_2: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': False, 'has_ip_in_url': False}
test_phish_1: {'has_url_shortener': False, 'has_long_domain': False, 'has_suspicious_tld': True, 'has_ip_in_url': False}
test_phish_2: {'has_url_shortener': True, 'has_long_domain': True, 'has_suspicious_tld': False, 'has_ip_in_url': False}

Тестиров

## 5. Тестирование threat_intelligence


In [22]:
import sqlite3

ti = ThreatIntelligence(str(TI_DB))

print("=" * 80)
print("Проверка наличия записей в базе данных")
print("=" * 80)

# Подключение к базе для проверки статистики
conn = sqlite3.connect(str(TI_DB))
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

# Подсчет записей в таблицах
cursor.execute("SELECT COUNT(*) as count FROM malicious_domains")
domains_count = cursor.fetchone()['count']

cursor.execute("SELECT COUNT(*) as count FROM malicious_ips")
ips_count = cursor.fetchone()['count']

print(f"\nКоличество записей в базе данных:")
print(f"  Домены: {domains_count}")
print(f"  IP-адреса: {ips_count}")

# Примеры записей доменов
if domains_count > 0:
    cursor.execute("SELECT domain, threat_type, date_added, source FROM malicious_domains LIMIT 5")
    domains_samples = cursor.fetchall()
    print(f"\nПримеры записей доменов:")
    for row in domains_samples:
        print(f"  Домен: {row['domain']}, Тип угрозы: {row['threat_type']}, "
              f"Дата добавления: {row['date_added']}, Источник: {row['source']}")
else:
    print("\nЗаписи доменов отсутствуют")

# Примеры записей IP
if ips_count > 0:
    cursor.execute("SELECT ip, threat_type, date_added, source FROM malicious_ips LIMIT 5")
    ips_samples = cursor.fetchall()
    print(f"\nПримеры записей IP-адресов:")
    for row in ips_samples:
        print(f"  IP: {row['ip']}, Тип угрозы: {row['threat_type']}, "
              f"Дата добавления: {row['date_added']}, Источник: {row['source']}")
else:
    print("\nЗаписи IP-адресов отсутствуют")

conn.close()

print("\n" + "=" * 80)
print("Тестирование проверки репутации на письмах")
print("=" * 80)

for name, email_content in test_emails.items():
    parsed = parse_email(email_content)
    domains = parsed.get('domains', [])[:3]
    ips = parsed.get('ips', [])[:3]
    
    for domain in domains:
        result = ti.check_domain_reputation(domain)
        if result['found']:
            print(f"{name}: domain={domain}, threat={result['threat_type']}, source={result['source']}")
    
    for ip in ips:
        result = ti.check_ip_reputation(ip)
        if result['found']:
            print(f"{name}: ip={ip}, threat={result['threat_type']}, source={result['source']}")


Проверка наличия записей в базе данных

Количество записей в базе данных:
  Домены: 2193
  IP-адреса: 7505

Примеры записей доменов:
  Домен: copperweide8.ru, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus
  Домен: auroragrat.ru, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus
  Домен: thistlehavn.ru, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus
  Домен: lake5p1rit.ru, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus
  Домен: lynchstem.ru, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus

Примеры записей IP-адресов:
  IP: 220.201.0.213, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus
  IP: 221.15.79.211, Тип угрозы: malware_download, Дата добавления: 2025-11-27T03:01:29.823526, Источник: URLhaus
  IP: 123.14.255.100, Т

## 6. Тестирование feature_extractor


In [25]:
print("=" * 80)
print("Тестирование модуля извлечения признаков")
print("=" * 80)

import numpy as np

# Инициализация компонентов
extractor = FeatureExtractor()
translator = Translator()

# Пайплайн обработки: parsed -> prepared_text -> translated -> url_analysis -> features
print("\nЭтап 1: Обработка писем и подготовка данных")
print("-" * 80)

parsed_emails = {}
translated_texts = []
url_analyses = {}

for name, email_content in test_emails.items():
    # Парсинг письма
    parsed = parse_email(email_content)
    parsed_emails[name] = parsed
    
    # Подготовка текста из parsed email (subject + body, очищенный от HTML)
    prepared_text = FeatureExtractor.prepare_text_from_parsed_email(parsed)
    
    # Перевод текста на английский (если требуется)
    translated = translator.translate_text(prepared_text)
    translated_texts.append(translated)
    
    # Анализ URL и доменов
    url_analysis = analyze_urls_and_domains(parsed)
    url_analyses[name] = url_analysis
    
    print(f"{name}: подготовлен текст длиной {len(prepared_text)}")

# Обучение векторизатора на переведенных текстах
print("\nЭтап 2: Обучение векторизатора")
print("-" * 80)
extractor.fit_vectorizer(translated_texts)
print(f"Векторизатор обучен на {len(translated_texts)} текстах")
print(f"Размер словаря: {len(extractor.tfidf_vectorizer.vocabulary_)}")

# Сбор синтетических признаков для обучения scaler
print("\nЭтап 3: Обучение scaler для синтетических признаков")
print("-" * 80)
synthetic_features_list = []

for name in test_emails.keys():
    parsed = parsed_emails[name]
    url_analysis = url_analyses[name]
    translated = translated_texts[list(test_emails.keys()).index(name)]
    
    # Извлечение синтетических признаков (до MinMax нормализации)
    quantitative_norm, _ = extractor.extract_quantitative_features(parsed)
    structural_norm, _ = extractor.extract_structural_features(parsed)
    binary = extractor.extract_binary_indicators(url_analysis, parsed)
    linguistic_norm, _ = extractor.extract_linguistic_features(translated)
    
    synthetic_features_array = np.concatenate([
        quantitative_norm, structural_norm, binary, linguistic_norm
    ])
    synthetic_features_list.append(synthetic_features_array)

# Обучение scaler
extractor.fit_scaler(synthetic_features_list)
print(f"Scaler обучен на {len(synthetic_features_list)} образцах")
print(f"Количество синтетических признаков: {synthetic_features_list[0].shape[0]}")

# Извлечение признаков для каждого письма
print("\nЭтап 4: Извлечение признаков для каждого письма")
print("-" * 80)

for name in test_emails.keys():
    parsed = parsed_emails[name]
    translated = translated_texts[list(test_emails.keys()).index(name)]
    url_analysis = url_analyses[name]
    
    # Полное извлечение признаков
    features = extractor.extract_all_features(parsed, translated, url_analysis)
    
    print(f"\n{name}:")
    print(f"  Количественные признаки: {features['synthetic_features']['quantitative']}")
    print(f"  Структурные признаки: {features['synthetic_features']['structural']}")
    print(f"  Бинарные индикаторы: {features['synthetic_features']['binary']}")
    print(f"  Лингвистические признаки: {features['synthetic_features']['linguistic']}")
    print(f"  Размер TF-IDF вектора: {features['tfidf_vector'].shape[0]}")
    print(f"  Размер полного вектора признаков: {features['feature_vector'].shape[0]}")
    print(f"  Размер синтетических признаков: {len(synthetic_features_list[list(test_emails.keys()).index(name)])}")

print("\n" + "=" * 80)
print("Тестирование завершено")
print("=" * 80)


Тестирование модуля извлечения признаков



Этап 1: Обработка писем и подготовка данных
--------------------------------------------------------------------------------
dataset_phishing_1: подготовлен текст длиной 468
dataset_phishing_2: подготовлен текст длиной 48
dataset_legit_1: подготовлен текст длиной 1079
dataset_legit_2: подготовлен текст длиной 3520
test_legit_1: подготовлен текст длиной 1022
test_legit_2: подготовлен текст длиной 941
test_phish_1: подготовлен текст длиной 1145
test_phish_2: подготовлен текст длиной 978

Этап 2: Обучение векторизатора
--------------------------------------------------------------------------------
Векторизатор обучен на 8 текстах
Размер словаря: 40

Этап 3: Обучение scaler для синтетических признаков
--------------------------------------------------------------------------------
Scaler обучен на 8 образцах
Количество синтетических признаков: 10

Этап 4: Извлечение признаков для каждого письма
--------------------------------------------------------------------------------

dataset_phis

## 7. Тестирование rules_engine


In [ ]:
print("=" * 80)
print("Тестирование на тестовых письмах")
print("=" * 80)

for name, email_content in test_emails.items():
    parsed = parse_email(email_content)
    headers = {k: parsed[k] for k in ['from', 'to', 'subject', 'auth_results', 'reply_to', 
                                       'return_path', 'received_headers', 'references']}
    header_analysis = analyze_headers(headers)
    ti_results = ti.check_reputation(parsed.get('domains', []), parsed.get('ips', []))
    result = evaluate_all_rules(header_analysis, parsed, ti_results)
    print(f"{name}: score={result['risk_score']}, level={result['risk_level']}, rules={len(result['triggered_rules'])}")

print("\n" + "=" * 80)
print("Тестирование на синтетических данных")
print("=" * 80)

from src.rules_engine import (
    check_authentication, check_domain_mismatch, check_threat_intelligence,
    check_reply_anomaly, check_dangerous_attachments
)

# Тесты правил
tests = [
    ('Аутентификация', check_authentication, [
        ({'spf_result': 'fail', 'dkim_result': 'pass', 'dmarc_result': 'pass'}, 'SPF fail'),
        ({'spf_result': 'pass', 'dkim_result': 'fail', 'dmarc_result': 'pass'}, 'DKIM fail'),
        ({'spf_result': 'pass', 'dkim_result': 'pass', 'dmarc_result': 'fail'}, 'DMARC fail'),
        ({'spf_result': 'pass', 'dkim_result': 'pass', 'dmarc_result': 'pass'}, 'Все пройдены')
    ]),
    ('Несоответствие доменов', check_domain_mismatch, [
        ({'from_domain': 'example.com', 'reply_to_domain': 'suspicious.com', 'return_path_domain': 'example.com'}, 'Reply-To не совпадает'),
        ({'from_domain': 'example.com', 'reply_to_domain': 'example.com', 'return_path_domain': 'suspicious.com'}, 'Return-Path не совпадает'),
        ({'from_domain': 'example.com', 'reply_to_domain': 'example.com', 'return_path_domain': 'example.com'}, 'Все совпадают')
    ]),
    ('Threat Intelligence', check_threat_intelligence, [
        ({'malicious_domains': ['malicious.com'], 'malicious_ips': []}, 'Домен в TI'),
        ({'malicious_domains': [], 'malicious_ips': ['192.168.1.100']}, 'IP в TI'),
        ({'malicious_domains': ['malicious.com'], 'malicious_ips': ['192.168.1.100']}, 'Домен и IP в TI'),
        ({'malicious_domains': [], 'malicious_ips': []}, 'Угрозы не найдены')
    ]),
    ('Аномалия ответа', check_reply_anomaly, [
        ({'has_re_without_references': True}, 'Re: без References'),
        ({'has_re_without_references': False}, 'Нормальное письмо')
    ]),
    ('Опасные вложения', check_dangerous_attachments, [
        ({'attachments': [{'name': 'file.exe'}]}, '.exe'),
        ({'attachments': [{'name': 'file.js'}]}, '.js'),
        ({'attachments': [{'name': 'file.pdf'}]}, '.pdf (безопасное)'),
        ({'attachments': []}, 'Без вложений')
    ])
]

for test_name, test_func, cases in tests:
    print(f"\n{test_name}:")
    for data, desc in cases:
        result = test_func(data)
        print(f"  {desc}: triggered={result['triggered']}, score={result['score']}")

# Комбинированный тест
print("\nКомбинированный тест:")
combined_result = evaluate_all_rules(
    {'spf_result': 'fail', 'dkim_result': 'fail', 'dmarc_result': 'fail',
     'from_domain': 'example.com', 'reply_to_domain': 'suspicious.com',
     'return_path_domain': 'suspicious.com',
     'has_re_without_references': True},
    {'attachments': [{'name': 'malware.exe'}]},
    {'malicious_domains': ['malicious.com'], 'malicious_ips': ['192.168.1.100']}
)
print(f"  Risk score: {combined_result['risk_score']}, level: {combined_result['risk_level']}")
print(f"  Правил сработало: {len(combined_result['triggered_rules'])}")
for rule in combined_result['triggered_rules']:
    print(f"    - {rule['rule_name']}: {rule['weight']} баллов")


Тестирование на тестовых письмах
dataset_phishing_1: score=25, level=LOW, rules=1
dataset_phishing_2: score=0, level=LOW, rules=0
dataset_legit_1: score=10, level=LOW, rules=1
dataset_legit_2: score=0, level=LOW, rules=0
test_legit_1: score=0, level=LOW, rules=0
test_legit_2: score=0, level=LOW, rules=0
test_phish_1: score=60, level=MEDIUM, rules=2
test_phish_2: score=55, level=MEDIUM, rules=3

Тестирование на синтетических данных

Аутентификация:
  SPF fail: triggered=True, score=15
  DKIM fail: triggered=True, score=15
  DMARC fail: triggered=True, score=10
  Все пройдены: triggered=False, score=0

Несоответствие доменов:
  Reply-To не совпадает: triggered=True, score=20
  Return-Path не совпадает: triggered=True, score=20
  Все совпадают: triggered=False, score=0

Threat Intelligence:
  Домен в TI: triggered=True, score=25
  IP в TI: triggered=True, score=25
  Домен и IP в TI: triggered=True, score=50
  Угрозы не найдены: triggered=False, score=0

Аномалия ответа:
  Re: без Referenc